## Impressions and Reviews 
- 코드는 간단하지만 대용량 데이터를 다루면서 import gc / gc.collect() garbage collector 작동으로 메모리 관리를 하는 점이 눈에 띄었다.  
- df.memory_usage(deep=True), astype('category')를 통해 메모리 사용현황을 파악하는 동시에 메모리사용을 줄이는 부분, float32, int32 사용에 의한 메모리 사용 감소도 가능하다

In [ ]:
"""
Ridge Regression on TfIDF of text features and One-Hot-Encoded Categoricals
"""

import pandas as pd
import numpy as np
import scipy

from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer

import gc

NUM_BRANDS = 2500
NAME_MIN_DF = 10
MAX_FEAT_DESCP = 50000

print("Reading in Data")

df_train = pd.read_csv('../input/train.tsv', sep='\t')
df_test = pd.read_csv('../input/test.tsv', sep='\t')

df = pd.concat([df_train, df_test], 0)
nrow_train = df_train.shape[0]
y_train = np.log1p(df_train["price"])

# 가비지 컬렉터 작동 - 해제 메모리 회수
del df_train
gc.collect() 

# dataframe 메모리 사용현황 : int나 float 변환, category 변환 으로 사용량 감소 가능
print(df.memory_usage(deep = True)) 

In [ ]:
df["category_name"] = df["category_name"].fillna("Other").astype("category")
df["brand_name"] = df["brand_name"].fillna("unknown")

# 가장 많은 2500위 브랜드까지만 사용
pop_brands = df["brand_name"].value_counts().index[:NUM_BRANDS] 
df.loc[~df["brand_name"].isin(pop_brands), "brand_name"] = "Other"

df["item_description"] = df["item_description"].fillna("None")
df["item_condition_id"] = df["item_condition_id"].astype("category")
df["brand_name"] = df["brand_name"].astype("category")

print(df.memory_usage(deep = True))

In [ ]:
print("Encodings")
count = CountVectorizer(min_df=NAME_MIN_DF)
X_name = count.fit_transform(df["name"])

print("Category Encoders")
unique_categories = pd.Series("/".join(df["category_name"].unique().astype("str")).split("/")).unique()
count_category = CountVectorizer()
X_category = count_category.fit_transform(df["category_name"])

print("Descp encoders")
count_descp = TfidfVectorizer(max_features = MAX_FEAT_DESCP, # 50000
                              ngram_range = (1,3),
                              stop_words = "english")
X_descp = count_descp.fit_transform(df["item_description"])

print("Brand encoders")
vect_brand = LabelBinarizer(sparse_output=True)
X_brand = vect_brand.fit_transform(df["brand_name"])

print("Dummy Encoders")
X_dummies = scipy.sparse.csr_matrix(pd.get_dummies(df[[
    "item_condition_id", "shipping"]], sparse = True).values) # sparse matrix class -> .toarray() 지원

X = scipy.sparse.hstack((X_dummies, 
                         X_descp,
                         X_brand,
                         X_category,
                         X_name)).tocsr()

print([X_dummies.shape, X_category.shape, 
       X_name.shape, X_descp.shape, X_brand.shape])

In [ ]:
X_train = X[:nrow_train]
model = Ridge(solver = "lsqr", fit_intercept=False)

print("Fitting Model")
model.fit(X_train, y_train)

X_test = X[nrow_train:]
preds = model.predict(X_test)

df_test["price"] = np.expm1(preds)
df_test[["test_id", "price"]].to_csv("submission_ridge.csv", index = False)